In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from tickers import getTickerList, getCandlePatternList
from datemodule import getDateRange
from indicators import ChannelBreakoutIndicator
from datasource import getFullData
import pandas

In [3]:
import talib
import pandas_ta as ta
from tqdm import tqdm

tickers = getTickerList(all=True)
tickerShortlist = {}
tickerShortlistData = {}

for i in tqdm(range(len(tickers))):
    ticker = tickers[i]
    resultDict = {}

    # Fetch Data
    startDate, endDate = getDateRange('6m')
    df = getFullData(ticker, startDate, endDate)
    
    candleIndex = len(df)-1
    
    # RSI
    df['RSI'] = ta.rsi(df.Close, length=14)

    # Candlestick Pattern Recognition
    op = df['Open']
    hi = df['High']
    lo = df['Low']
    cl = df['Close']
    for pattern in getCandlePatternList(all=False):
        sig = getattr(talib, pattern)(op, hi, lo, cl)[candleIndex]
        if sig != 0:
            resultDict[pattern] = sig
    
    # Channel Breakout Indicator Signal
    cbIndicator = ChannelBreakoutIndicator(df, ticker)
    cbIndicator.calculate(40)
    cbSignal = cbIndicator.getBuySell()[-1]
    if (cbSignal != ''):
        resultDict["ChannelBreakoutIndicator"] = cbSignal

    # Collect Signals
    if (len(resultDict) != 0):
        resultDict["RSI"] = df['RSI'][candleIndex]
        tickerShortlistData[ticker] = cbIndicator
        tickerShortlist[ticker] = resultDict

100%|██████████| 96/96 [01:44<00:00,  1.08s/it]


In [7]:
shortlisted = pandas.DataFrame(tickerShortlist).transpose()
entries = []
for index, row in shortlisted.iterrows():
    entries.append(row.count())
shortlisted["ENTRIES"] = entries
shortlisted = shortlisted.sort_values(by='ENTRIES', ascending=False).drop(['ENTRIES'], axis=1)
shortlisted

,CDLMARUBOZU,RSI,CDLHANGINGMAN,CDLENGULFING,ChannelBreakoutIndicator,CDLHARAMI,CDLMORNINGSTAR
DIVISLAB,NaN,66.681755,NaN,NaN,BUY,-100,NaN
UPL,NaN,43.831289,NaN,NaN,SELL,100,NaN
AXISBANK,NaN,59.396586,NaN,100.0,NaN,NaN,100.0
IOC,100.0,76.435763,NaN,NaN,NaN,NaN,NaN
SIEMENS,NaN,54.638621,NaN,NaN,SELL,NaN,NaN
TCS,NaN,59.660992,NaN,100.0,NaN,NaN,NaN
DMART,NaN,53.56889,NaN,NaN,SELL,NaN,NaN
SRF,NaN,36.439549,NaN,100.0,NaN,NaN,NaN
HCLTECH,NaN,61.471851,NaN,-100.0,NaN,NaN,NaN
NYKAA,NaN,54.291457,NaN,NaN,NaN,100.0,NaN


In [8]:
for ticker in shortlisted.index:
    print(tickerShortlist[ticker])
    tickerShortlistData[ticker].showIndicator(len(tickerShortlistData[ticker].df.index)-1, 40)

{'CDLHARAMI': -100, 'ChannelBreakoutIndicator': 'BUY', 'RSI': 66.68175537061359}
0.9994222906156514 0.9956019285953134


{'CDLHARAMI': 100, 'ChannelBreakoutIndicator': 'SELL', 'RSI': 43.8312893631536}
0.3909399926959394 0.16474464579901313


{'CDLENGULFING': 100, 'CDLMORNINGSTAR': 100, 'RSI': 59.39658648729068}
0.9235105701540115 1.0


{'CDLMARUBOZU': 100, 'RSI': 76.43576297056902}
1.0 1.0


{'ChannelBreakoutIndicator': 'SELL', 'RSI': 54.638621118286544}
0.9527306959296833 0.02083131603773285


...

{'CDLENGULFING': 100, 'RSI': 68.02568862254674}
0.9320660390764998 0.9861452597409583


{'ChannelBreakoutIndicator': 'BUY', 'RSI': 70.4151521574868}
0.18693553410154917 0.5820378665261432


{'ChannelBreakoutIndicator': 'BUY', 'RSI': 75.53676172597906}
0.7659849247469654 0.9217681732632189


{'CDLENGULFING': -100, 'RSI': 54.46647295762593}
1.0 1.0


{'CDLHANGINGMAN': -100, 'RSI': 59.18483232950907}
1.0 0.9285593527319022
